Public Companies Scraping

Get all the public companies information in America by scraping from NYSE.

In [ ]:
import requests
import bs4
import pandas as pd
import json
import numpy
import regex as re

In [ ]:
payloadHeader = {
    "Content-Type": "application/json",
}
payloadData = {
    "instrumentType":"EQUITY",
    "pageNumber":1,
    "sortColumn":"NORMALIZED_TICKER",
    "sortOrder":"ASC",
    "maxResultsPerPage":1000000,
    "filterToken":"",
}
# get the json requested
url = "https://www.nyse.com/api/quotes/filter"
html = requests.post(url, data=json.dumps(payloadData), headers=payloadHeader)
listing = json.loads(html.text)
# scrap the companies information
fileObject = open('listing.json', 'w')
fileObject.write(html.text)
fileObject.close()
company = list()
for i in range(6446):
    ticker = listing[i]["symbolTicker"].replace('.', '').split("-")[0]
    name = listing[i]["instrumentName"]
    url = f"https://finance.yahoo.com/quote/{ticker}/profile"
    html = requests.get(url)
    response = bs4.BeautifulSoup(html.content,"lxml")
    print(f"{i} {ticker}")
    try:
        website = response.find("div", class_="Mb(25px)").find_all("a")[1].get_text()
        company.append((ticker, name, website))
    except:
        print("fail")
# output the results
listing = pd.DataFrame(company, columns=['ticker','name','website']).drop_duplicates('website')
listing = listing.reset_index(drop=True)
# listing.to_csv("listing.csv")